# Copyright Netherlands eScience Center <br>
** Function     : Packing netCDF for the surface wind fields (u&v 10m) from ERA-Interim** <br>
** Author       : Yang Liu ** <br>
** First Built  : 2018.12.04 ** <br>
** Last Update  : 2018.12.04 ** <br>
Description     : This notebook aims to pack the surface wind fields from ERA-Interim.<br>
Return Values   : netCDF4 <br>
Caveat          : <br>

In [1]:
import numpy as np
import scipy as sp
import time as tttt
from netCDF4 import Dataset,num2date
import os

In [2]:
################################   Input zone  ######################################
# specify starting and ending time
start_year = 1979
end_year = 2017
# specify data path
# ERAI 3D fields on pressure level
datapath_radiation = '/home/ESLT0068/WorkFlow/Core_Database_AMET_OMET_reanalysis/ERAI/regression/surface/uv10m'
# specify output path for figures
output_path = '/home/ESLT0068/WorkFlow/Core_Database_AMET_OMET_reanalysis/ERAI/regression'
####################################################################################

In [3]:
def var_key_retrieve(datapath, year):
    # get the path to each datasets
    print ("Start retrieving datasets %d (y)" % (year))
    # The shape of each variable is (241,480)
    datapath_full = os.path.join(datapath, 'surface_erai_monthly_075_%d_uv10m.nc' % (year))
    # get the variable keys
    var_key = Dataset(datapath_full)

    print ("Retrieving datasets successfully and return the variable key!")
    return var_key

In [4]:
def create_netcdf_point (pool_u10, pool_v10, pool_si10, output_path):
    print ('*******************************************************************')
    print ('*********************** create netcdf file*************************')
    print ('*******************************************************************')
    #logging.info("Start creating netcdf file for the 2D fields of ERAI at each grid point.")
    # wrap the datasets into netcdf file
    # 'NETCDF3_CLASSIC', 'NETCDF3_64BIT', 'NETCDF4_CLASSIC', and 'NETCDF4'
    data_wrap = Dataset(os.path.join(output_path, 'surface_erai_monthly_regress_1979_2017_wind.nc'),'w',format = 'NETCDF4')
    # create dimensions for netcdf data
    year_wrap_dim = data_wrap.createDimension('year',Dim_year)
    month_wrap_dim = data_wrap.createDimension('month',Dim_month)
    lat_wrap_dim = data_wrap.createDimension('latitude',Dim_latitude)
    lon_wrap_dim = data_wrap.createDimension('longitude',Dim_longitude)
    # create coordinate variable
    year_wrap_var = data_wrap.createVariable('year',np.int32,('year',))
    month_wrap_var = data_wrap.createVariable('month',np.int32,('month',))
    lat_wrap_var = data_wrap.createVariable('latitude',np.float32,('latitude',))
    lon_wrap_var = data_wrap.createVariable('longitude',np.float32,('longitude',))
    # create the actual 4d variable
    u10_wrap_var = data_wrap.createVariable('u10',np.float64,('year','month','latitude','longitude'),zlib=True)
    v10_wrap_var = data_wrap.createVariable('v10',np.float64,('year','month','latitude','longitude'),zlib=True)
    si10_wrap_var = data_wrap.createVariable('si10',np.float64,('year','month','latitude','longitude'),zlib=True) 
    # global attributes
    data_wrap.description = 'Monthly mean 2D fields of ERA-Interim on surface level'
    # variable attributes
    lat_wrap_var.units = 'degree_north'
    lon_wrap_var.units = 'degree_east'

    u10_wrap_var.units = 'm/s'
    v10_wrap_var.units = 'm/s'
    si10_wrap_var.units = 'm/s'

    u10_wrap_var.long_name = '10m zonal wind'
    v10_wrap_var.long_name = '10m meridional wind'
    si10_wrap_var.long_name = '10m wind speed'

    # writing data
    lat_wrap_var[:] = latitude
    lon_wrap_var[:] = longitude
    month_wrap_var[:] = index_month
    year_wrap_var[:] = period

    u10_wrap_var[:] = pool_u10
    v10_wrap_var[:] = pool_v10
    si10_wrap_var[:] = pool_si10

    # close the file
    data_wrap.close()
    print ("The generation of netcdf files for fields on surface is complete!!")

In [5]:
def retriver(key):
    print ('Extract synoptic mean fields.')
    u10 = var_key.variables['u10'][:]
    v10 = var_key.variables['v10'][:]
    si10 = var_key.variables['si10'][:]

    return u10, v10, si10

In [6]:
if __name__=="__main__":
    ####################################################################
    ######  Create time namelist matrix for variable extraction  #######
    ####################################################################
    # date and time arrangement
    # namelist of month and days for file manipulation
    namelist_month = ['01','02','03','04','05','06','07','08','09','10','11','12']
    # index of months
    period = np.arange(start_year,end_year+1,1)
    index_month = np.arange(1,13,1)
    ####################################################################
    ######       Extract invariant and calculate constants       #######
    ####################################################################
    # get invariant from benchmark file
    Dim_year = len(period)
    Dim_month = len(index_month)
    Dim_latitude = 241
    Dim_longitude = 480
    #############################################
    #####   Create space for stroing data   #####
    #############################################
    # data pool
    pool_u10 = np.zeros((Dim_year,Dim_month,Dim_latitude,Dim_longitude),dtype = float)
    pool_v10 = np.zeros((Dim_year,Dim_month,Dim_latitude,Dim_longitude),dtype = float)
    pool_si10 = np.zeros((Dim_year,Dim_month,Dim_latitude,Dim_longitude),dtype = float)
    latitude = np.zeros(Dim_latitude,dtype=float)
    longitude = np.zeros(Dim_longitude,dtype=float)
    # loop for calculation
    for i in period:
        # get the key of each variable
        var_key = var_key_retrieve(datapath_radiation,i)
        latitude = var_key.variables['latitude'][:]
        longitude = var_key.variables['longitude'][:]
        u10, v10, si10 = retriver(var_key)
        pool_u10[i-1979,:,:,:] = u10
        pool_v10[i-1979,:,:,:] = v10
        pool_si10[i-1979,:,:,:] = si10
    ####################################################################
    ######                 Data Wrapping (NetCDF)                #######
    ####################################################################
    create_netcdf_point(pool_u10, pool_v10, pool_si10, output_path)
    print ('Packing 2D fields of ERA-Interim on surface level is complete!!!')
    print ('The output is in sleep, safe and sound!!!')

Start retrieving datasets 1979 (y)
Retrieving datasets successfully and return the variable key!
Extract synoptic mean fields.
Start retrieving datasets 1980 (y)
Retrieving datasets successfully and return the variable key!
Extract synoptic mean fields.
Start retrieving datasets 1981 (y)
Retrieving datasets successfully and return the variable key!
Extract synoptic mean fields.
Start retrieving datasets 1982 (y)
Retrieving datasets successfully and return the variable key!
Extract synoptic mean fields.
Start retrieving datasets 1983 (y)
Retrieving datasets successfully and return the variable key!
Extract synoptic mean fields.
Start retrieving datasets 1984 (y)
Retrieving datasets successfully and return the variable key!
Extract synoptic mean fields.
Start retrieving datasets 1985 (y)
Retrieving datasets successfully and return the variable key!
Extract synoptic mean fields.
Start retrieving datasets 1986 (y)
Retrieving datasets successfully and return the variable key!
Extract synopt